# Visualizing CASP Data

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
def make_training_example(lines):
    """
    Takes lines read from file and created a training example.
    """
    example = {}
    
    example['id'] = lines[1].rstrip()
    example['primary'] = lines[3].rstrip()
    
    pssm = []
    for i in range(5, 26):
        pssm.append([float(x) for x in lines[i].split()])
    example['pssm'] = pssm
    
    tertiary = []
    for i in range(27, 30):
        tertiary.append([float(x) for x in lines[i].split()])
    example['tertiary'] = tertiary
    
    example['mask'] = lines[31].rstrip()
    
    return example

In [9]:
with open('data/text_based/training/training_30', 'r') as raw_data:
    lines = []
    for line in raw_data:
        lines.append(line)
        if len(lines) == 33:
            example = make_training_example(lines)
            break
len(example)

5

In [30]:
def get_tertiary_matrix(tertiary, i):
    x = tertiary[0][3 * i], tertiary[0][3 * i + 1], tertiary[0][3 * i + 2]
    y = tertiary[1][3 * i], tertiary[1][3 * i + 1], tertiary[1][3 * i + 2]
    z = tertiary[2][3 * i], tertiary[2][3 * i + 1], tertiary[2][3 * i + 2]
    return x, y, z

In [31]:
count = 0
for i in range(len(example['mask'])):
    if example['mask'][i] == "+":
        print(example['primary'][i])
        print(get_tertiary_matrix(example['tertiary'], i))
        count += 1
        if count > 10:
            break

K
((2841.8, 2873.4, 2919.7), (-864.7, -957.9, -877.0), (-6727.1, -6616.3, -6496.2))
Y
((2883.1, 2935.1, 2956.3), (-924.6, -875.1, -995.3), (-6377.8, -6251.2, -6159.3))
Y
((3002.6, 3022.5, 3148.1), (-1105.5, -1233.7, -1302.1), (-6219.0, -6152.1, -6207.9))
H
((3266.2, 3392.6, 3461.0), (-1264.3, -1319.4, -1419.0), (-6159.5, -6212.8, -6118.0))
A
((3523.8, 3582.0, 3706.7), (-1519.8, -1629.4, -1678.7), (-6177.4, -6103.0, -6174.7))
N
((3822.3, 3952.8, 4000.8), (-1626.4, -1659.3, -1800.9), (-6133.5, -6192.3, -6157.1))
S
((3960.1, 3999.5, 4003.8), (-1899.0, -2040.1, -2118.9), (-6238.3, -6219.8, -6350.4))
T
((4057.7, 4066.6, 4026.3), (-2240.4, -2329.8, -2476.2), (-6342.7, -6458.4, -6430.6))
E
((4012.5, 3993.2, 3848.5), (-2513.0, -2653.1, -2701.7), (-6303.0, -6263.5, -6286.9))
E
((3815.2, 3682.6, 3569.7), (-2820.8, -2878.8, -2782.0), (-6235.6, -6260.6, -6223.2))
D
((3465.5, 3361.3, 3264.5), (-2783.4, -2681.1, -2700.6), (-6305.3, -6305.1, -6188.5))
